# Teams and Hotels

Playing around with a backtracking algorithm to solve the problem of assigning teams to hotels during an athletics event. 

In [113]:
import pandas
import numpy as np

In [114]:
teams = pandas.read_csv('C:/Users/jsmulter/Desktop/test.csv', sep=';') \
    .assign(total = lambda df: df['single'] + df['double'])

In [115]:
hotels = pandas.read_csv('hotels.csv', sep=';')

In [124]:
def solve(hotels, teams):
    # keep track of state in the dataframes themselves0
    teams_hotels = teams \
        .sort_values(['total'], ascending=False) \
        .assign(hotel=None).reset_index(drop=True)
        
    hotels_rooms = hotels \
        .assign(remaining_single = lambda df: df['capacity_single']) \
        .assign(remaining_double = lambda df: df['capacity_double'])
    
    if not place_team(hotels_rooms, teams_hotels, 0):
        return 'No solution exists.'
    else:
        return (hotels_rooms, teams_hotels)
    
def cycle_hotels(hotels, offset):
    # First hotel is tried first (should be filled to capacity)
    new_indexes = [0]
    # Round-robin through the other hotels
    curr = offset + 1
    for _ in range(len(hotels) - 1):
        new_indexes.append(curr)
        curr += 1
        if curr == len(hotels):
            curr = 1
    return hotels.reindex(new_indexes)
    
def place_team(hotels, teams_hotels, team_idx):
    if team_idx >= len(teams_hotels):
        # Exit condition -- all teams have been placed
        return True
    
    t = teams_hotels.loc[team_idx]

    cycled_hotels = cycle_hotels(hotels, team_idx % len(hotels))

    for hotel_idx, hotel in cycled_hotels.iterrows():
        if can_place_team(t, hotel):
            teams_hotels.loc[team_idx, 'hotel'] = hotel['name']
            hotels.loc[hotel_idx, 'remaining_single'] -= t['single']
            hotels.loc[hotel_idx, 'remaining_double'] -= t['double']
            
            if place_team(hotels, teams_hotels, team_idx + 1):
                return True
            
            # Backtrack
            teams_hotels.loc[team_idx, 'hotel'] = None
            hotels.loc[hotel_idx, 'remaining_single'] += t['single']
            hotels.loc[hotel_idx, 'remaining_double'] += t['double']
            
    return False

def can_place_team(team, hotel):
    return team['hotel'] is None and \
           hotel['remaining_single'] >= team['single'] and \
           hotel['remaining_double'] >= team['double']
    


In [125]:
hotels, teams = solve(hotels, teams)